In [348]:
# load reviews
import os
import kagglehub
import pandas as pd
import numpy as np
import swifter

from utils.geocode_utils import get_country_code, memory

In [349]:
# load the reviews
@memory.cache
def load_reviews():
  # fetch reviews from kaggle
  path = kagglehub.dataset_download("christopheiv/winemagdata130k")
  fname = "winemag-data-130k-v2.csv"
  return pd.read_csv(os.path.join(path, fname), index_col=0)

In [350]:
# load the reviews
reviews = load_reviews()

reviews["code"] = reviews["country"].swifter.apply(get_country_code)
reviews[["winery", "province", "country", "code"]].head()

Pandas Apply:   0%|          | 0/129971 [00:00<?, ?it/s]

,winery,province,country,code
0,Nicosia,Sicily & Sardinia,Italy,IT
1,Quinta dos Avidagos,Douro,Portugal,PT
2,Rainstorm,Oregon,US,US
3,St. Julian,Michigan,US,US
4,Sweet Cheeks,Oregon,US,US


In [351]:
# extract the winery locations to geolocate
location_cols = ['winery', 'region_1', 'region_2', 'province', 'country', 'code']
wineries = np.unique(reviews.winery.dropna())
locations = reviews.query("winery in @wineries").copy()[location_cols].drop_duplicates()
print(
    f"{locations.shape[0] - len(wineries):,d}",
    "duplicate winery names in different locations",
)
locations.info()
locations.head()

13,659 duplicate winery names in different locations
<class 'pandas.core.frame.DataFrame'>
Index: 30416 entries, 0 to 129952
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   winery    30416 non-null  object
 1   region_1  26697 non-null  object
 2   region_2  10023 non-null  object
 3   province  30389 non-null  object
 4   country   30389 non-null  object
 5   code      30389 non-null  object
dtypes: object(6)
memory usage: 1.6+ MB


,winery,region_1,region_2,province,country,code
0,Nicosia,Etna,NaN,Sicily & Sardinia,Italy,IT
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal,PT
2,Rainstorm,Willamette Valley,Willamette Valley,Oregon,US,US
3,St. Julian,Lake Michigan Shore,NaN,Michigan,US,US
4,Sweet Cheeks,Willamette Valley,Willamette Valley,Oregon,US,US


### Location Cleanup

**Locations where location fields duplicate the same information OR contain the word `Other`**

In [352]:
# cleanup helper
def clean_duplicate_location_field_value(keep, clean, df=locations, preview=False):
  same_value = df[clean] == df[keep]
  value_with_other = (df[clean].notna()) & ((df[clean] == 'Other')|(df[clean] == df[keep] + ' Other'))
  if preview:
    print(np.sum(same_value), f"locations['{clean}'] values are the same as locations['{keep}']")
    print(np.sum(value_with_other), f"locations['{clean}'] values contain 'Other'")
    print(f"unique locations['{clean}'] values that contain 'Other':", np.unique(locations[value_with_other][clean]))
    print()
  else:
    df[clean] = df[clean].mask(same_value | value_with_other, None)


In [353]:
# preview the changes
clean_duplicate_location_field_value(keep='country', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='province', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='region_1', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='country', clean='region_1', preview=True)
clean_duplicate_location_field_value(keep='province', clean='region_1', preview=True)
clean_duplicate_location_field_value(keep='country', clean='province', preview=True)

0 locations['region_2'] values are the same as locations['country']
0 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': []

0 locations['region_2'] values are the same as locations['province']
1119 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': ['California Other' 'New York Other' 'Oregon Other' 'Washington Other']

882 locations['region_2'] values are the same as locations['region_1']
983 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': ['California Other' 'New York Other' 'Oregon Other' 'Washington Other']

87 locations['region_1'] values are the same as locations['country']
0 locations['region_1'] values contain 'Other'
unique locations['region_1'] values that contain 'Other': []

1988 locations['region_1'] values are the same as locations['province']
6 locations['region_1'] values contain 'Other'
unique locations['region_

In [354]:
# cleanup
clean_duplicate_location_field_value(keep='country', clean='region_2')
clean_duplicate_location_field_value(keep='province', clean='region_2')
clean_duplicate_location_field_value(keep='region_1', clean='region_2')
clean_duplicate_location_field_value(keep='country', clean='region_1')
clean_duplicate_location_field_value(keep='province', clean='region_1')
clean_duplicate_location_field_value(keep='country', clean='province')

In [355]:
# verify the changes
clean_duplicate_location_field_value(keep='country', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='province', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='region_1', clean='region_2', preview=True)
clean_duplicate_location_field_value(keep='country', clean='region_1', preview=True)
clean_duplicate_location_field_value(keep='province', clean='region_1', preview=True)
clean_duplicate_location_field_value(keep='country', clean='province', preview=True)

0 locations['region_2'] values are the same as locations['country']
0 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': []

0 locations['region_2'] values are the same as locations['province']
0 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': []

0 locations['region_2'] values are the same as locations['region_1']
0 locations['region_2'] values contain 'Other'
unique locations['region_2'] values that contain 'Other': []

0 locations['region_1'] values are the same as locations['country']
0 locations['region_1'] values contain 'Other'
unique locations['region_1'] values that contain 'Other': []

0 locations['region_1'] values are the same as locations['province']
0 locations['region_1'] values contain 'Other'
unique locations['region_1'] values that contain 'Other': []

0 locations['province'] values are the same as locations['country']
0 locations['province'] values contain 'Othe

**Locations with the word `Vin` in `region_1`**

Some indicate city or wine growing regions, like `Vin Santo di Montepulciano` or `Vin de Pays des Côtes de Gascogne`, but others are not regions like `Vin Mousseux` or `Vin Santo del Chianti Classico`. Need to remove the meaningless values, and reduce the remaining ones to their regions.

In [356]:
region_1_with_Vin = np.unique(locations[(locations['region_1'].notna())&(locations['region_1'].str.contains('Vin '))]['region_1'])
region_1_with_Vin

array(['Vin Doux Naturel Rasteau', 'Vin Mousseux', 'Vin Pétillant',
       'Vin Santo del Chianti', 'Vin Santo del Chianti Classico',
       'Vin Santo del Chianti Rufina', 'Vin Santo di Carmignano',
       'Vin Santo di Montepulciano', 'Vin de France', 'Vin de Liqueur',
       'Vin de Pays Cité de Carcassonne', 'Vin de Pays Var',
       "Vin de Pays d'Oc", 'Vin de Pays de France',
       "Vin de Pays de L'Aude", "Vin de Pays de L'Herault",
       'Vin de Pays de Montferrand', 'Vin de Pays de Vaucluse',
       "Vin de Pays de l'Atlantique", "Vin de Pays de l'Ile de Beauté",
       'Vin de Pays de la Haute Vallée du Gassac',
       'Vin de Pays de la Méditerranée', 'Vin de Pays des Alpilles',
       'Vin de Pays des Coteaux de Bessilles', 'Vin de Pays des Cévennes',
       'Vin de Pays des Côtes Catalanes',
       'Vin de Pays des Côtes de Gascogne', 'Vin de Pays des Maures',
       'Vin de Pays des Portes de Méditerranée',
       'Vin de Pays du Comté Tolosan', 'Vin de Pays du Gard',
 

Remove the meaningless regions

In [357]:
regions_to_remove = [
  'Vin Doux Naturel Rasteau', 'Vin Mousseux', 'Vin Pétillant',
  'Vin Santo del Chianti', 'Vin Santo del Chianti Classico',
  'Vin Santo del Chianti Rufina', 'Vin de France', 'Vin de Liqueur',
  'Vin de Table Francais'
]
locations[locations['region_1'].isin(regions_to_remove)]

,winery,region_1,region_2,province,country,code
82,Lionel Osmin & Cie,Vin de France,NaN,None,France,FR
731,Kiwi Cuvée,Vin de France,NaN,None,France,FR
780,Castello d'Albola,Vin Santo del Chianti Classico,NaN,Tuscany,Italy,IT
911,Frédéric Brouca,Vin de France,NaN,None,France,FR
1121,Domaine Rotier,Vin de Liqueur,NaN,None,France,FR
...,...,...,...,...,...,...
122397,Grandissime,Vin de France,NaN,None,France,FR
126713,Domaine du Grand Cros,Vin Mousseux,NaN,None,France,FR
127051,Fat Bastard,Vin de France,NaN,None,France,FR
128902,Château de Brigue,Vin Mousseux,NaN,None,France,FR


In [358]:
locations['region_1'] = locations['region_1'].mask(locations['region_1'].isin(regions_to_remove), None)
locations[locations['region_1'].isin(regions_to_remove)]

,winery,region_1,region_2,province,country,code


Reduce the `region_1` field to the region name only

In [359]:
# italian wines
locations['region_1'] = locations['region_1'].mask((locations['region_1'].notna())&(locations['region_1'].str.startswith('Vin Santo di ')), locations['region_1'].str.replace('Vin Santo di ', ''))
locations[(locations['region_1'].notna())&locations['region_1'].str.startswith('Vin Santo di ')]

,winery,region_1,region_2,province,country,code


In [360]:
# french wines
locations['region_1'] = locations['region_1'].mask((locations['region_1'].notna())&(locations['region_1'].str.startswith('Vin de ')), locations['region_1'].str.replace('Vin de ', ''))
locations[(locations['region_1'].notna())&locations['region_1'].str.startswith('Vin de ')]

,winery,region_1,region_2,province,country,code


In [361]:
# verify the cleanup
locations[(locations['region_1'].notna())&(locations['region_1'].str.contains('Vin '))]

,winery,region_1,region_2,province,country,code


In [362]:
np.unique(locations['region_1'].dropna())

array(['Abruzzo', 'Adelaida District', 'Adelaide', ...,
       'Yorkville Highlands', 'Yountville', 'Zonda Valley'],
      shape=(1170,), dtype=object)

In [363]:
locations.head()

,winery,region_1,region_2,province,country,code
0,Nicosia,Etna,NaN,Sicily & Sardinia,Italy,IT
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal,PT
2,Rainstorm,Willamette Valley,None,Oregon,US,US
3,St. Julian,Lake Michigan Shore,NaN,Michigan,US,US
4,Sweet Cheeks,Willamette Valley,None,Oregon,US,US


**Remove provinces that are not actual province names**

In [364]:
# mask = locations.swifter.apply(lambda row: type(row['province']) == str and type(row['country']) == str and row['province'] != 'Port' and row['province'] in row['country'], axis=1, result_type='reduce')

mask = locations.swifter.apply(lambda row: row.country in row.province if row.province is not None and row.country is not None and type(row.province) == str and type(row.country) else False, axis=1)
locations[mask]
# [
#   # True if ((row[1] is not None and row[2] is not None) and (row[2] in row[1])) else False
#   row[1] is not None and row[2] is not None and type(row[1]) == str and type(row[2]) == str and row[2] in row[1]
#   for row in locations[['province', 'country']].itertuples()
# ]

Pandas Apply:   0%|          | 0/30416 [00:00<?, ?it/s]

,winery,region_1,region_2,province,country,code
5,Tandem,Navarra,NaN,Northern Spain,Spain,ES
18,Pradorey,Ribera del Duero,NaN,Northern Spain,Spain,ES
38,Feudi di San Marzano,Puglia,NaN,Southern Italy,Italy,IT
61,Podere dal Nespoli,Romagna,NaN,Central Italy,Italy,IT
72,Grifalco,Aglianico del Vulture,NaN,Southern Italy,Italy,IT
...,...,...,...,...,...,...
129350,Vigneti Villabella,Delle Venezie,NaN,Northeastern Italy,Italy,IT
129507,Endrizzi,Trento,NaN,Northeastern Italy,Italy,IT
129760,Luigi Maffini,Paestum,NaN,Southern Italy,Italy,IT
129850,Macchialupa,Campania,NaN,Southern Italy,Italy,IT


In [365]:
locations.loc[mask, 'province'] = None
locations[mask].head()

,winery,region_1,region_2,province,country,code
5,Tandem,Navarra,NaN,None,Spain,ES
18,Pradorey,Ribera del Duero,NaN,None,Spain,ES
38,Feudi di San Marzano,Puglia,NaN,None,Italy,IT
61,Podere dal Nespoli,Romagna,NaN,None,Italy,IT
72,Grifalco,Aglianico del Vulture,NaN,None,Italy,IT


**Remove provinces with `&` in name**

In [366]:
mask = (locations.province.notna()) & (locations.province.str.contains('&'))
print(np.unique(locations[mask].province))
pd.concat(
  [
    locations[locations.province == p].sample(3)
    for p in np.unique(locations[mask].province)
  ]
)

['Casablanca & Leyda Valleys' 'Sicily & Sardinia']


,winery,region_1,region_2,province,country,code
82894,Carmen,NaN,NaN,Casablanca & Leyda Valleys,Chile,CL
109876,Kingston Family,NaN,NaN,Casablanca & Leyda Valleys,Chile,CL
116329,Montes,NaN,NaN,Casablanca & Leyda Valleys,Chile,CL
24,Canicattì,Sicilia,NaN,Sicily & Sardinia,Italy,IT
11859,Maggio Vini,Sicilia,NaN,Sicily & Sardinia,Italy,IT
89541,Danzante,Sicilia,NaN,Sicily & Sardinia,Italy,IT


In [367]:
locations.loc[mask, 'province'] = None
locations[mask].sample(5)

,winery,region_1,region_2,province,country,code
119347,FiàNobile,Sicilia,NaN,None,Italy,IT
49942,Botter,Sicilia,NaN,None,Italy,IT
74575,Spadina,Sicilia,NaN,None,Italy,IT
996,Caruso & Minini,Sicilia,NaN,None,Italy,IT
4523,Biondi,Etna,NaN,None,Italy,IT


**Check for Duplicate Locations**

In [368]:
print('after cleanup, there are', locations.duplicated().sum(), 'duplicate locations')
locations = locations.drop_duplicates()

after cleanup, there are 98 duplicate locations


In [369]:
print('there are', locations.duplicated().sum(), 'duplicates left')
locations.describe()

there are 0 duplicates left


,winery,region_1,region_2,province,country,code
count,30318,24401,8022,25842,30291,30291
unique,16757,1170,13,379,43,43
top,Louis Latour,Napa Valley,Sonoma,California,US,US
freq,43,903,1934,8185,10868,10868


## Search Query Augmentation
Use the least location terms where possible.

In [370]:
# progress indicator
progress = lambda: print(
  ' '.join(
    [f'{locations['q'].notna().sum()/locations.shape[0]:.1%}', 'locations have geocode query expressions']
  )
)

### Create query string

**Wineries that have no `region_1`, `region_2`, or `province` fields**

In [371]:
mask = (
  (locations['winery'].notna())
    &((locations['region_1'].isna()))
    &((locations['region_2'].isna()))
    &((locations['province'].isna()))
)

# create the query column and set the query to the winery name
locations = locations.assign(q=locations['winery'].where(mask, None))

In [372]:
# verify that these wineries were set
assert locations.loc[mask,'q'].isna().sum() == 0
locations[mask].head()

,winery,region_1,region_2,province,country,code,q
77,Yalumba,None,NaN,None,Australia,AU,Yalumba
82,Lionel Osmin & Cie,None,NaN,None,France,FR,Lionel Osmin & Cie
232,Angove's,None,NaN,None,Australia,AU,Angove's
400,Cantine Maschio,None,NaN,None,Italy,IT,Cantine Maschio
731,Kiwi Cuvée,None,NaN,None,France,FR,Kiwi Cuvée


In [373]:
# and that others are not
assert locations.loc[~mask,'q'].notna().sum() == 0
locations[~mask].head()

,winery,region_1,region_2,province,country,code,q
0,Nicosia,Etna,NaN,None,Italy,IT,None
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal,PT,None
2,Rainstorm,Willamette Valley,None,Oregon,US,US,None
3,St. Julian,Lake Michigan Shore,NaN,Michigan,US,US,None
4,Sweet Cheeks,Willamette Valley,None,Oregon,US,US,None


In [374]:
# check on progress
progress()

1.6% locations have geocode query expressions


#### Helper functions to create query expressions from select location fields

In [375]:
# helper function that builds mask based on which columns to select
def mask_builder(cols: list[str]):
  mask = locations.q.isna()
  mask &= locations.winery.notna()
  mask &= locations.region_1.notna() if 'region_1' in cols else locations.region_1.isna()
  mask &= locations.region_2.notna() if 'region_2' in cols else locations.region_2.isna()
  mask &= locations.province.notna() if 'province' in cols else locations.province.isna()
  return mask

In [376]:
# helper function that builds the geocode query epression from the selected location fields
def query_builder(cols: list[str], df: pd.DataFrame=locations):
  mask = mask_builder(cols)
  return df.loc[mask].swifter.apply(lambda row: ', '.join(row[c] for c in cols), axis=1)

In [377]:
# helper function
def create_query_expression(cols, df=locations):
  df.loc[mask, 'q'] = query_builder(cols, df)
  progress()
  

**Remaining wineries with `province` and no `region_1` or `region_2`**

In [378]:
cols = ['province']

# verify mask
mask = mask_builder(cols)
assert locations.region_1[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].notna().sum() == 0
locations[mask].head()

,winery,region_1,region_2,province,country,code,q
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal,PT,None
7,Trimbach,None,NaN,Alsace,France,FR,None
8,Heinz Eifel,NaN,NaN,Rheinhessen,Germany,DE,None
9,Jean-Baptiste Adam,None,NaN,Alsace,France,FR,None
11,Leon Beyer,None,NaN,Alsace,France,FR,None


In [379]:
create_query_expression(cols)

# verify query expressions
assert locations.region_1[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()

Pandas Apply:   0%|          | 0/5432 [00:00<?, ?it/s]

19.5% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal,PT,Douro
7,Trimbach,None,NaN,Alsace,France,FR,Alsace
8,Heinz Eifel,NaN,NaN,Rheinhessen,Germany,DE,Rheinhessen
9,Jean-Baptiste Adam,None,NaN,Alsace,France,FR,Alsace
11,Leon Beyer,None,NaN,Alsace,France,FR,Alsace


**Remaining wineries with `region_1` and no `province` or `region_2`**

In [380]:
cols = ['region_1']

# verify mask
mask = mask_builder(cols)
assert locations.province[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].notna().sum() == 0

# set the query expression
create_query_expression(cols)

# verify query expressions
assert locations.province[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()


Pandas Apply:   0%|          | 0/3991 [00:00<?, ?it/s]

32.7% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q
0,Nicosia,Etna,NaN,None,Italy,IT,Etna
5,Tandem,Navarra,NaN,None,Spain,ES,Navarra
6,Terre di Giurfo,Vittoria,NaN,None,Italy,IT,Vittoria
13,Masseria Setteporte,Etna,NaN,None,Italy,IT,Etna
16,Felix Lavaque,Cafayate,NaN,None,Argentina,AR,Cafayate


**Remaining wineries with `region_2` and no `province` or `region_1`**

In [381]:
cols = ['region_2']

# verify mask
mask = mask_builder(cols)
assert locations.province[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].notna().sum() == 0

# set the query expression
create_query_expression(cols)

# verify query expressions
assert locations.province[mask].notna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()

32.7% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q


**Remaining wineries with `province` and `region_1` and no `region_2`**

In [382]:
cols = ['province', 'region_1']

# verify mask
mask = mask_builder(cols)
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].notna().sum() == 0

# set the query expression
create_query_expression(cols)

# verify query expressions
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()

Pandas Apply:   0%|          | 0/12388 [00:00<?, ?it/s]

73.5% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q
2,Rainstorm,Willamette Valley,None,Oregon,US,US,"Oregon, Willamette Valley"
3,St. Julian,Lake Michigan Shore,NaN,Michigan,US,US,"Michigan, Lake Michigan Shore"
4,Sweet Cheeks,Willamette Valley,None,Oregon,US,US,"Oregon, Willamette Valley"
14,Mirassou,Central Coast,None,California,US,US,"California, Central Coast"
17,Gaucho Andino,Mendoza,NaN,Mendoza Province,Argentina,AR,"Mendoza Province, Mendoza"


**Remaining wineries with `province` and `region_2` and no `region_1`**

In [383]:
cols = ['province', 'region_2']

# verify mask
mask = mask_builder(cols)
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].notna().sum() == 0

# set the query expression
create_query_expression(cols)

# verify query expressions
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].notna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()

73.5% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q


**Remaining wineries with all 3 fields `province`, `region_1` and `region_2`, populated**

In [384]:
cols = ['province', 'region_1', 'region_2']

# verify mask
mask = mask_builder(cols)
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].isna().sum() == 0
assert locations.q[mask].notna().sum() == 0

# set the query expression
create_query_expression(cols)

# verify query expressions
assert locations.province[mask].isna().sum() == 0 and locations.region_1[mask].isna().sum() == 0 and locations.region_2[mask].isna().sum() == 0
assert locations.q[mask].isna().sum() == 0
locations[mask].head()

Pandas Apply:   0%|          | 0/8022 [00:00<?, ?it/s]

100.0% locations have geocode query expressions


,winery,region_1,region_2,province,country,code,q
10,Kirkland Signature,Napa Valley,Napa,California,US,US,"California, Napa Valley, Napa"
12,Louis M. Martini,Alexander Valley,Sonoma,California,US,US,"California, Alexander Valley, Sonoma"
23,Bianchi,Paso Robles,Central Coast,California,US,US,"California, Paso Robles, Central Coast"
25,Castello di Amorosa,Sonoma Coast,Sonoma,California,US,US,"California, Sonoma Coast, Sonoma"
29,Clarksburg Wine Company,Clarksburg,Central Valley,California,US,US,"California, Clarksburg, Central Valley"


**Use direct http requests to geocode the wineries**

In [385]:
import requests

def geocode_by_winery_and_country(name, code):
  url = lambda endpoint: f'https://api.mapbox.com/{endpoint}'
  endpoint = 'search/searchbox/v1/forward'
  params = dict(q=name, country=code, poi_category='winery', language='en', access_token='LETMEIN')
  r = requests.get(url(endpoint), params=params)
  print(r.status_code, r.headers)
  return r.json()

In [386]:
# geocode_by_winery_and_country('100 Percent Wine', 'US')

## Tests

In [387]:
import unittest


class GeocodeWineryTestCase(unittest.TestCase):

    def test_invalid_request(self):
        print('not implemented')
        
    def test_not_authenticated(self):
        print('not implemented')
        
    def test_lookup_valid_winery(self):
        print("not implemented")

    def test_lookup_unknown_winery(self):
        print("not implemented")

    def test_multiple_matches(self):
        print("not implemented")

    def test_throttling_error(self):
        print("not implemented")


if __name__ == "__main__":
    unittest.main(argv=[""], verbosity=2, exit=False)

test_invalid_request (__main__.GeocodeWineryTestCase.test_invalid_request) ... ok
test_lookup_unknown_winery (__main__.GeocodeWineryTestCase.test_lookup_unknown_winery) ... ok
test_lookup_valid_winery (__main__.GeocodeWineryTestCase.test_lookup_valid_winery) ... ok
test_multiple_matches (__main__.GeocodeWineryTestCase.test_multiple_matches) ... ok
test_not_authenticated (__main__.GeocodeWineryTestCase.test_not_authenticated) ... ok
test_throttling_error (__main__.GeocodeWineryTestCase.test_throttling_error) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


not implemented
not implemented
not implemented
not implemented
not implemented
not implemented
